In [1]:
%matplotlib inline
import torch

In [2]:
model = torch.hub.load('pytorch/vision', 'resnet50', pretrained=True)

Using cache found in /home/yina/.cache/torch/hub/pytorch_vision_master


In [3]:
# Download an example image from the pytorch website
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

In [4]:
# sample execution (requires torchvision)
from PIL import Image
from torchvision import transforms, datasets, models
input_image = Image.open(filename)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)
# Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
print(torch.nn.functional.softmax(output[0], dim=0))
print(output[0].numpy().argmax())

tensor([-1.0941e+00, -3.0869e-01, -5.2834e-01, -1.0523e+00, -5.9791e-01,
        -2.5296e-01, -7.7640e-01,  5.3440e-01,  4.1246e-01, -4.7661e-01,
        -7.6271e-01, -7.7440e-01, -3.6173e-01, -8.5679e-01, -9.7189e-01,
        -4.4033e-01, -5.2238e-01, -7.0338e-02, -7.7410e-02, -3.6458e-01,
        -1.0817e+00, -6.5617e-01, -1.1831e+00, -9.8250e-02, -8.7666e-01,
        -7.5350e-01, -8.7502e-01, -7.1281e-01, -5.8139e-01, -3.1549e-01,
        -9.1589e-01, -8.6432e-01, -6.6319e-01, -5.4284e-01, -1.8587e-01,
        -6.2590e-01,  4.8753e-01, -6.6665e-01, -3.9458e-01,  1.6061e-01,
        -5.5537e-01, -8.0956e-01, -7.5005e-01, -3.2449e-01, -2.2420e-01,
        -4.6225e-01, -5.4693e-01, -4.1038e-01, -1.1333e+00, -7.1230e-01,
        -3.7443e-01,  3.6199e-01, -3.6375e-01, -4.0716e-01, -3.7833e-01,
        -1.0858e+00, -6.4781e-01, -1.1347e+00, -3.9610e-01, -1.4917e-01,
         6.8407e-01, -8.5677e-02, -1.6376e-01, -2.0467e-01, -7.9047e-01,
        -3.7820e-01, -4.8365e-01, -3.8252e-01, -7.9

In [5]:
from zoo.common.nncontext import init_spark_on_local, init_spark_on_yarn
from zoo.pipeline.api.net.torch_net import TorchNet
import numpy as np
import os
hadoop_conf_dir = os.environ.get('HADOOP_CONF_DIR')

if hadoop_conf_dir:
    num_executors = 2
    num_cores_per_executor = 4
    sc = init_spark_on_yarn(
    hadoop_conf=hadoop_conf_dir,
    conda_name=os.environ["ZOO_CONDA_NAME"], # The name of the created conda-env
    num_executor=num_executors,
    executor_cores=num_cores_per_executor,
    executor_memory="10g",
    driver_memory="2g",
    driver_cores=1)
else:
    num_executors = 1
    num_cores_per_executor = 4
    sc = init_spark_on_local(cores = 4, conf = {"spark.driver.memory": "10g"})

Prepending /home/yina/anaconda3/envs/face-gen/lib/python3.6/site-packages/bigdl/share/conf/spark-bigdl.conf to sys.path
Adding /home/yina/anaconda3/envs/face-gen/lib/python3.6/site-packages/zoo/share/lib/analytics-zoo-bigdl_0.8.0-spark_2.4.3-0.6.0-SNAPSHOT-jar-with-dependencies.jar to BIGDL_JARS
Prepending /home/yina/anaconda3/envs/face-gen/lib/python3.6/site-packages/zoo/share/conf/spark-analytics-zoo.conf to sys.path
Current pyspark location is : /home/yina/anaconda3/envs/face-gen/lib/python3.6/site-packages/pyspark/__init__.py
Start to getOrCreate SparkContext
/home/yina/anaconda3/envs/face-gen/bin/python


/home/yina/anaconda3/envs/face-gen/bin/python

Successfully got a SparkContext


In [6]:
net = TorchNet.from_pytorch(model, [1, 3, 224, 224])

creating: createTorchNet


In [13]:
from torchvision import transforms, datasets, models
val_loader = torch.utils.data.DataLoader(
        datasets.ImageFolder("..", transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])),
        batch_size=8, shuffle=False,
        num_workers=1, pin_memory=True)

In [14]:
for inputs, label in val_loader:
    output = net.predict(inputs.numpy(), distributed=True).collect()
    index = [o.argmax() for o in output]
    print(index)

[463]
